In [1]:
# Supress unnecessary warnings so that presentation looks clean
import warnings
warnings.filterwarnings('ignore')

import pandas
import numpy as np
import xgboost as xgb
from sklearn.cross_validation import KFold
from sklearn.metrics import mean_absolute_error
import zipfile
from sklearn import cross_validation, metrics


zip_ref = zipfile.ZipFile("train.csv.zip", 'r')
zip_ref.extractall()
zip_ref.close()
zip_ref2 = zipfile.ZipFile("test.csv.zip", 'r')
zip_ref2.extractall()
zip_ref2.close()


train_data = pandas.read_csv("train.csv")
test_data = pandas.read_csv("test.csv")
test_data['loss'] = np.nan


joined = pandas.concat([train_data, test_data],ignore_index = True)
del train_data,test_data

cat_feature = [n for n in joined.columns if n.startswith('cat')]

for column in cat_feature:
        joined[column] = pandas.factorize(joined[column].values, sort=True)[0]
        
del cat_feature



train_data = joined[joined['loss'].notnull()]
test_data = joined[joined['loss'].isnull()]
del joined

ids = test_data['id']


shift = 200
train_data["loss"] = np.log(train_data["loss"]+shift)
target = train_data['loss']
train_data.drop(['id','loss'],1,inplace=True)
test_data.drop(['id','loss'],1,inplace=True)


/home/demonicode/.local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/demonicode/.local/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [16]:
def eval_loss(estimator,train, label):
    
    x= mean_absolute_error(np.exp(estimator.predict(train))-shift, np.exp(label)-shift)
    return -x

In [17]:
d_train_full = xgb.DMatrix(train_data, label=target)
d_test = xgb.DMatrix(test_data)

In [18]:
def modelfit(alg,train_data,target,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(train_data, label=target)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            feval=eval_loss, early_stopping_rounds=early_stopping_rounds)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(train_data, target,eval_metric=eval_loss)

In [19]:
xgb1 = XGBClassifier(n_estimators=10,
 learning_rate =0.1,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'reg:linear',
 scale_pos_weight=1,
 seed=2016)



In [12]:
modelfit(xgb1, train_data[:100], target[:100])

AttributeError: 'DMatrix' object has no attribute 'exp'

In [ ]:
clf = xgb1.fit(train_data[:100],target[:100])

In [ ]:
#from sklearn.metrics import make_scorer
#err = make_scorer(eval_loss,greater_is_better=False)

param_test1 = {
 'max_depth':[12,13],
 'min_child_weight':[1,2]
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'reg:linear', nthread=1, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring=eval_loss,n_jobs=4,iid=False, cv=5)
gsearch1.fit(train_data[:],target[:])
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

Process PoolWorker-26:
Process PoolWorker-27:
Process PoolWorker-28:
Traceback (most recent call last):
Process PoolWorker-25:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/demonicode/anaconda2/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/home/demonicode/anaconda2/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/home/demonicode/anaconda2/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/home/demonicode/anaconda2/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
    self.run()
    self.run()
    self.run()
  File "/home/demonicode/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/home/demonicode/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/home/demonicode/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/home/d

In [79]:
from sklearn.metrics import make_scorer
err = make_scorer(eval_loss,greater_is_better=False)

param_test1 = {
 'max_depth':[12,13],
 'min_child_weight':[1,2]
}
gsearch1 = GridSearchCV(estimator = xgb1 , param_grid =param_test1, scoring = err)
gsearch1.fit(train_data[:100],target[:100])

print (gsearch1.grid_scores_)
print (gsearch1.best_params_)
print (gsearch1.best_score_)

AttributeError: 'numpy.ndarray' object has no attribute 'get_label'